In [ ]:
import datetime as dt
import random
from functools import reduce
from itertools import chain

period_in_seconds = 120
period_delta = dt.timedelta(seconds=period_in_seconds)

start = dt.datetime(2017,1,1,0,0,0,0)
end  = dt.datetime(2017,1,31,0,0,0,0)

In [ ]:
# Generates all the requests in one period made by all devices (1:1)
def generate_requests(devices, start, period_in_seconds: int)->list:
    return [ start + dt.timedelta(seconds=random.randrange(period_in_seconds)) for _ in range (0, devices) ]

In [ ]:
# Generates a sliding window that moves within the requests (or resolutions, depending which is greater)
# Chops a big timespam into several periods and returns each of these periods as (start,end)
def window_generator(start:dt.datetime, period:dt.timedelta, end:dt.datetime)->(dt.datetime, dt.datetime):
    while start < end:
        a = start
        b = start + period
        yield (a, b)
        start = start + period

In [ ]:
# Functional toolbox

def compose(*funcs):
    def _comp(a, b):
        def _app(*n):
            return b(*a(*n))
        return _app
    return reduce(_comp, funcs)

# Given a f(x), generates a f'(a,b) = (f(a), b)
def apply_left(f):
    def applyleft(a, b):
        return f(a), b
    return applyleft

# Given a f(x), generates a f'(a,b) = (a, f(b))
def apply_right(f):
    def applyright(a, b):
        return a, f(b)
    return applyright


In [ ]:
# Returns a filter function that returns True if request is within window. 
# A request is charaterized by its timestamp
def timefilter_builder(window_start:dt.datetime, window_end:dt.datetime):
    def timefilter(request):
        return window_start <= request and request < window_end
    return timefilter

#
def split_requests_builder(timefilter):
    def split_requests(requests):
        reqs_in_window = filter(timefilter, requests)
        reqs_not_in_window = filter(lambda x: not timefilter(x),requests)
        return reqs_in_window, reqs_not_in_window
    return split_requests

# Returns 
def push_requests_builder(period_time):
    def push_requests(requests):
        return map(lambda x:x+period_time,requests)
    return push_requests


"""
def linear_growth_builder(factor):
    def linear_growth(requests)
        return chain(requests, 
"""

timefilter = timefilter_builder(start, end)

# split -> in, out 
# push -> in

# requests -> (filtered, remaining) 
filter_requests = split_requests_builder(timefilter)
# (filtered, remaining) ->  (new remaining)
push_requests = compose(apply_left(push_requests_builder(period_delta)),chain)

# Flatmap-like:
totalreqs = iter([])
requests = generate_requests(10, start, period_in_seconds)


filtered, remaining = filter_requests(requests)
totalreqs = chain(filtered, totalreqs)
newrequests = push_requests(filtered, remaining)


list(totalreqs)




In [ ]:
"""
def split_and_push(requests, timefilter, push_time:dy.timedelta):
    (reqs_in, reqs_out) = split_requests(requests)
    reqs_pushed = push_requests(reqs_in, push_time)
    return reqs_pushed, reqs_out
"""


# Apply filter function to our requests distribution, pushes forward filtered requests
# one period ahead (kick to the future) and returns new requests distribution.
"""
def requests_filter(timefilter, request_distribution:list, period:dt.timedelta)->(iter,iter):
    reqs_in_window = filter(timefilter, request_distribution)
    #reqs_in_next_window = map(lambda x:x+period, reqs_in_window)
    #new_requests = list(filter(lambda x: not timefilter(x),request_distribution)) + list(reqs_in_next_window)   
    new_requests = chain(push_requests(), 
                         filter(lambda x: not timefilter(x),request_distribution))

    return new_requests, reqs_in_window
"""

In [ ]:
"""
requests = generate_requests(10, start, period_in_seconds)

my_windows = window_generator(start, period_delta, start+period_delta*5)

my_filters = map(lambda x:timefilter_builder(*x),my_windows)

total_reqs = []
for filtro in my_filters:
    (requests, res) = requests_filter(filtro, requests, period_delta)
    total_reqs += list(res)
    
print(len(total_reqs))
total_reqs

"""

requests = generate_requests(10, start, period_in_seconds)

my_windows = window_generator(start, period_delta, start+period_delta*5)

my_filters = map(lambda x:timefilter_builder(*x),my_windows)

def filter_and_push(requests_accumulator, timefilter):
    (requests, accumulated_requests) = requests_accumulator
    
    (new_requests, filtered_reqs) = requests_filter(timefilter, requests, period_delta)
    return new_requests, accumulated_requests + filtered_reqs
    
# f1: myfilter(window_1), f2: myfilter(window_2),...
# filter_and_push(filter_and_push(filter_and_push(filter_and_push(requests, f1), f2), f3), f4)
# accumulated_requests sums all intermediate changes 
# final_requests are the remaining requests outside the windows (future)
(final_request, acumulated_request) = reduce(filter_and_push, my_filters, (requests, []))

acumulated_request

In [ ]:
reduce?